In [1]:
#Modules

import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from dateutil import parser
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt



def get_sma(prices, rate):
    return prices.rolling(rate).mean()



def get_bollinger_bands(prices, rate):
    sma = get_sma(prices, rate)
    std = prices.rolling(rate).std()
    bollinger_up = sma + std * 2 # Calculate top band
    bollinger_down = sma - std * 2 # Calculate bottom band
    return bollinger_up, bollinger_down, sma


def rsi(df, periods, ema = True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
	    # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi


def atr(data, periods):
    high_low = data['high'] - data['low']
    high_close = np.abs(data['high'] - data['close'].shift())
    low_close = np.abs(data['low'] - data['close'].shift())

    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    atr = true_range.rolling(periods).sum()/periods
    return atr

# dfmr= pd.DataFrame(columns=['IV','DV','DV2','DV3','DV4','DV5','DV6','date']) #Dv is returns from indicator until the eod of the strategy.
# dfwtw6= pd.DataFrame(columns=['IV','DV','DV2','DV3','DV4','DV5','DV6','date']) # DV is returns form the xth half hour until the eod. 
# dfwtw65= pd.DataFrame(columns=['IV','DV2','DV3','DV4','DV5','DV6','date'])
# dfwtw7= pd.DataFrame(columns=['IV','DV2','DV3','DV4','DV5','DV6','date'])
# dfhhr= pd.DataFrame(columns=['IV','DV','DV2','DV3','DV4','DV5','DV6','date']) #Dv is last half hour returns of every day. 
# #Can use the dfmr for buy and hold strategy.
# dkljsdf = Path(r"C:\Users\david\Documents\universiteit\VU\Finance and Technology\Thesis")
# dfmr.to_csv(Path(dkljsdf, 'dfmr.csv'))
# dfwtw6.to_csv(Path(dkljsdf, 'dfwtw6.csv'))
# dfwtw65.to_csv(Path(dkljsdf, 'dfwtw65.csv'))
# dfwtw7.to_csv(Path(dkljsdf, 'dfwtw7.csv'))
# dfhhr.to_csv(Path(dkljsdf, 'dfhhr.csv'))


In [154]:
#Load in the data. feeder model
def fiveminohlcmaker(stockname,initialfilename,dateevent) -> str:   
    df = pd.read_csv(initialfilename+'.csv') 
    df['DATETIME2']= np.nan
    df['datetime']= np.nan
    df['DATETIME2']= df["DATE"].astype(str) +' '+df["TIME_M"].astype(str)
    df['datetime'] = pd.to_datetime(df['DATETIME2'], format= '%Y%m%d %H:%M:%S.%f')
    # print(df)
    df = df[df['SYM_SUFFIX'].isna()]
    # print(df.tail(50))
    # print(df.groupby(pd.Grouper(key='datetime', axis=0, freq='D')).sum())b 
    df2 = df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).sum()
    df2['open']=np.nan
    df2['high']=np.nan
    df2['low']=np.nan
    df2['close']=np.nan

    df2['open']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).PRICE.first()
    df2['high']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).PRICE.max()
    df2['low']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).PRICE.min()
    df2['close']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).PRICE.last()
    #try filling na forward. 
    if 'SYM_SUFFIX' in df2.columns:
        df2.drop(columns=['DATE','SYM_SUFFIX', 'PRICE'], axis=1, inplace=True)
    else: 
        df2.drop(columns=['DATE', 'PRICE'], axis=1, inplace=True)
    df2.dropna(axis=0, inplace=True)
    df2 = df2.apply(lambda x: x.asfreq('5min')).between_time('9:30', '16:00', include_start=True, include_end=False)
    dftomerge = df2.iloc[:,-1]
    dftomerge = dftomerge[dftomerge.notnull().groupby(dftomerge.index.normalize()).transform('sum') > 0]
    dftomerge = dftomerge.fillna(method='ffill')
    dftomerge = dftomerge.fillna(method='bfill')
    # print('dftomerge')
    # print(dftomerge.iloc[0:].head(50))
    df2.dropna(axis=0, inplace=True)
    df2 = df2.merge(dftomerge, left_index=True, right_index=True, how='right')
    # print(len(df2))

    # print(df2.iloc[220:].head(50))
    # print(len(df2))
    df2 = df2.drop(columns=['close_x'])
    df2 = df2.rename(columns={"close_y": "close"})
    df2 = df2.fillna(method='ffill')
    df2 = df2.fillna(method='bfill')
    df2['return']=np.nan
    # df2['return'] = np.log(df2['close'].shift(0))-np.log(df2['close'].shift(1))
    # df2['return'][0] = np.log(df2['close'][0])-np.log(df2['open'][0])
    df2['return'] = (df2['close'].shift(0)-df2['close'].shift(1))/df2['close'].shift(1)
    df2['return'][0] = (df2['close'][0]-df2['open'][0])/df2['open'][0]
    # print('print(df2)')
    # print(df2.iloc[220:].head(50))
    # df2.to_csv(r'C:\Users\david\Documents\universiteit\VU\Finance and
    # df2.to_csv(r'C:\Users\david\Documents\universiteit\VU\Finance and Technology\Thesis\+name'.csv', index=False)
    
    # print(df2.tail(50))

    #Calculate Bollinger Bands
    closing_prices = df2['close']
    rate =  50 #2 weeks bollinger bands just to test
    bollinger_up, bollinger_down, sma = get_bollinger_bands(closing_prices, rate)
    #Add it to the dataframe, can use the rate in order to know where to place it in the dataframe. 
    df2['bollinger_down'] = np.nan
    df2['sma'] = np.nan
    df2['bollinger_up'] = np.nan
    df2['bollinger_down'] = bollinger_down
    df2['sma'] = sma
    df2['bollinger_up'] = bollinger_up


    #Calculate RSI 
    rsi1 = rsi(df2,14)
    df2['rsi'] = np.nan
    df2['rsi'] = rsi1
    #Add it to the datafrma

    #Calculate ATR
    atr1 = atr(df2,14)
    df2['atr1'] = np.nan
    df2['atr1'] = atr1
    #add it to the datafrma. 

    
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=df2.index,
                open=df2['open'], high=df2['high'],
                low=df2['low'], close=df2['close'])) 
                
    # fig.add_trace(go.Scatter(x = closing_prices.index, y = bollinger_up))
    # fig.add_trace(go.Scatter(x = closing_prices.index, y = bollinger_down))
    # fig.add_trace(go.Scatter(x = closing_prices.index, y = sma))                 

    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()

    # fig2 = go.Figure()
    # fig2.add_trace(go.Scatter(x = closing_prices.index, y = rsi1))
    # fig.update_layout(xaxis_rangeslider_visible=False)
    # fig2.show()

    # fig3 = go.Figure()
    # fig3.add_trace(go.Scatter(x = closing_prices.index, y = atr1))
    # fig.update_layout(xaxis_rangeslider_visible=False)
    # fig3.show()

    #Make indicator colomns with for loops
    df2['bbI'] = np.abs
    df2['rsiI'] = np.abs
    df2['atrI'] = np.abs

    #Cut the dataframe until where the data is the day of the event.
    df2 = df2.loc[df2.index >= dateevent]

    #reset index
    df2 = df2.reset_index()

    #input indicators in rows as 1. 
    for i in range(len(df2)):
        if df2.low[i] <= df2.bollinger_down[i]:
            df2['bbI'].iloc[i:]=1
            print(i, 'bb works')
            break
    for i in range(len(df2)):
        if df2.high[i] >= df2.bollinger_up[i]:
            df2['bbI'].iloc[i:]=1
            print(i, 'bb works')
            break        
    for i in range(len(df2)):
        if df2.rsi[i] <= 30:
            df2['rsiI'].iloc[i:]=1
            print(i, 'rsi works')
            break        
    for i in range(len(df2)):
        if df2.rsi[i] >= 70:
            df2['rsiI'].iloc[i:]=1
            print(i, 'rsi works')
            break


    df2['atrhigh'] = 0.0    
    for i in range(len(df2)):
        if df2.atr1[i] > df2.atrhigh.max():
            df2['atrhigh'].iloc[i:]= df2.atr1[i]
        elif df2.atr1[i] <= 0.750*df2.atrhigh.max():
            df2['atrI'].iloc[i:] = 1
            print(i, 'ATR works')
            break
    
    # print(df2['bbI'])
    
    # print(df2.iloc[-1])
    df2['IndicatorMeanReversion'] = np.nan
    for i in range(len(df2)):
        if df2['bbI'][i] ==  df2['rsiI'][i] == df2['atrI'][i] == 1:
            df2['IndicatorMeanReversion'].iloc[i:] = 1
            break 
    
    
    df2['datetime'] = pd.to_datetime(df2['datetime'], format= '%Y%m%d %H:%M:%S.%f')
    # print(df2)
    #indicator starting
    indicatornum = len(df2.loc[df2['IndicatorMeanReversion'] != 1])
    # print(indicatornum)
        #check of dit geen verkeerde time frame by one day geeft. 
    df3 = df2.loc[(df2['datetime'] >= df2['datetime'][0]) & (df2['datetime'] <(df2['datetime'][indicatornum]))].reset_index(drop=True) #independent variable
    df9 = df2.loc[(df2['datetime'] >= df2['datetime'][indicatornum]) & (df2['datetime'] <=(df2['datetime'].iloc[-391]))].reset_index(drop=True)
    df4 = df2.loc[(df2['datetime'] >= df2['datetime'][indicatornum]) & (df2['datetime'] <=(df2['datetime'].iloc[-313]))].reset_index(drop=True)#dpeendend variables. #These should always have 78 because want the end of the first day.
    df5 = df2.loc[(df2['datetime'] >= df2['datetime'][indicatornum]) & (df2['datetime'] <=(df2['datetime'].iloc[-235]))].reset_index(drop=True)
    df6 = df2.loc[(df2['datetime'] >= df2['datetime'][indicatornum]) & (df2['datetime'] <=(df2['datetime'].iloc[-157]))].reset_index(drop=True)
    df7 = df2.loc[(df2['datetime'] >= df2['datetime'][indicatornum]) & (df2['datetime'] <=(df2['datetime'].iloc[-79]))].reset_index(drop=True)
    df8 = df2.loc[(df2['datetime'] >= df2['datetime'][indicatornum]) & (df2['datetime'] <=(df2['datetime'].iloc[-1]))].reset_index(drop=True)
    # print(df3)
    # print(df9)
    # print(df4)
    # print(df5)
    # print(df6)
    # print(df7)
    # print(df8)
    
    
    # dfmr= pd.DataFrame(columns=['IV','DV','DV2','DV3','DV4','DV5','DV6']) 
    # print(df9.dtypes)
    dfmr = pd.read_csv('dfmr.csv') 
    IVreturns = ((1 + df3['return']).cumprod() - 1)[indicatornum-1]    
    DVreturn1 = ((1 + df9['return']).cumprod() - 1)[len(df9)-1] #cumprod until end of the first  day.
    DVreturn2 = ((1 + df4['return']).cumprod() - 1)[len(df4)-1]# cumprod until the end of the second day enzovoorts. 
    DVreturn3 = ((1 + df5['return']).cumprod() - 1)[len(df5)-1]
    DVreturn4 = ((1 + df6['return']).cumprod() - 1)[len(df6)-1]
    DVreturn5 = ((1 + df7['return']).cumprod() - 1)[len(df7)-1]
    DVreturn6 = ((1 + df8['return']).cumprod() - 1)[len(df8)-1]
    adding = {'IV':IVreturns, 'DV': DVreturn1,'DV2':DVreturn2,'DV3':DVreturn3,'DV4':DVreturn4,'DV5':DVreturn5,'DV6': DVreturn6, 'date': dateevent}
    dfmr = dfmr.append(adding, ignore_index=True)


    p = Path(r"C:\Users\david\Documents\universiteit\VU\Finance and Technology\Thesis")
    dfmr.to_csv(Path(p, 'dfmr.csv'), index=False)


    #Now make the dataset for with the wind. 

    #Need returns for first 6 hours(half hour before close), first 6,5 hours(until close) and 7 hours(half hour after next day open close). 

    #Need half hour dataframe with prices and returns. 
    dfwtw = df.groupby(pd.Grouper(key='datetime', axis=0, freq='30min')).sum()
    dfwtw['open']=np.nan
    dfwtw['high']=np.nan
    dfwtw['low']=np.nan
    dfwtw['close']=np.nan
    dfwtw['open']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='30min')).PRICE.first()
    dfwtw['high']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='30min')).PRICE.max()
    dfwtw['low']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='30min')).PRICE.min()
    dfwtw['close']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='30min')).PRICE.last()
    # print(len(dfwtw))
    if 'SYM_SUFFIX' in dfwtw.columns:
        dfwtw.drop(columns=['DATE','SYM_SUFFIX', 'PRICE'], axis=1, inplace=True)
    else: 
        dfwtw.drop(columns=['DATE', 'PRICE'], axis=1, inplace=True)
    dfwtw.dropna(axis=0, inplace=True)
    dfwtw = dfwtw.apply(lambda x: x.asfreq('30min')).between_time('9:30', '16:00', include_start=True, include_end=False)
    dftomerge2 = dfwtw.iloc[:,-1]
    dftomerge2 = dftomerge2[dftomerge2.notnull().groupby(dftomerge2.index.normalize()).transform('sum') > 0]
    dftomerge2 = dftomerge2.fillna(method='ffill')
    dftomerge2 = dftomerge2.fillna(method='bfill')
    # print('dftomerge2')
    # print(dftomerge2.iloc[0:].head(50))
    dfwtw.dropna(axis=0, inplace=True)
    dfwtw = dfwtw.merge(dftomerge2, left_index=True, right_index=True, how='right')
    # print(len(dfwtw))

    # print(dfwtw.iloc[30:].head(50))
    # print(len(dfwtw))
    dfwtw = dfwtw.drop(columns=['close_x'])
    dfwtw = dfwtw.rename(columns={"close_y": "close"})
    dfwtw = dfwtw.fillna(method='ffill')
    dfwtw = dfwtw.fillna(method='bfill')
    dfwtw['return']=np.nan
    # dfwtw['return'] = np.log(dfwtw['close'].shift(0))-np.log(dfwtw['close'].shift(1))
    # dfwtw['return'][0] = np.log(dfwtw['close'][0])-np.log(dfwtw['open'][0])
    dfwtw['return'] = (dfwtw['close'].shift(0)-dfwtw['close'].shift(1))/dfwtw['close'].shift(1)
    dfwtw['return'][0] = (dfwtw['close'][0]-dfwtw['open'][0])/dfwtw['open'][0]
    # dfwtw.to_csv(r'C:\Users\david\Documents\universiteit\VU\Finance and
    dfwtw = dfwtw.loc[dfwtw.index >= dateevent]
    # print(len(dfwtw))
    #reset index
    dfwtw = dfwtw.reset_index()
    dfwtw['datetime'] = pd.to_datetime(dfwtw['datetime'], format= '%Y%m%d %H:%M:%S.%f')
    # print('print(dfwtw)')
    # print(dfwtw)
    #We can use this dataframe for every one of the last strategies. 
    # start with dataframe for 6 hour iv. 
    dfwtw6  = pd.read_csv('dfwtw6.csv') 
    timeindicator = 12 #6 hours
    df10 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][0]) & (dfwtw['datetime'] <(dfwtw['datetime'][timeindicator]))] #independent variable
    df11 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator]) & (dfwtw['datetime'] <(dfwtw['datetime'].iloc[13]))].reset_index(drop=True)
    df12 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-53]))].reset_index(drop=True) #dpeendend variables. #These should always have 13 because want the end of the first day.
    df13 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-40]))].reset_index(drop=True)
    df14 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-27]))].reset_index(drop=True)
    df15 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-14]))].reset_index(drop=True)
    df16 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-1]))].reset_index(drop=True)
    sixIVreturns = ((1 + df10['return']).cumprod() - 1).iloc[timeindicator-1]    
    sixDVreturn1 = ((1 + df11['return']).cumprod() - 1).iloc[-1] #cumprod until end of the first  day.
    sixDVreturn2 = ((1 + df12['return']).cumprod() - 1).iloc[-1]# cumprod until the end of the second day enzovoorts. 
    sixDVreturn3 = ((1 + df13['return']).cumprod() - 1).iloc[-1]
    sixDVreturn4 = ((1 + df14['return']).cumprod() - 1).iloc[-1]
    sixDVreturn5 = ((1 + df15['return']).cumprod() - 1).iloc[-1]
    sixDVreturn6 = ((1 + df16['return']).cumprod() - 1).iloc[-1]

    adding2 = {'IV':sixIVreturns, 'DV': sixDVreturn1,'DV2':sixDVreturn2,'DV3':sixDVreturn3,'DV4':sixDVreturn4,'DV5':sixDVreturn5,'DV6': sixDVreturn6, 'date': dateevent}
    dfwtw6 = dfwtw6.append(adding2, ignore_index=True) 
    dfwtw6.to_csv(Path(p, 'dfwtw6.csv'), index=False)

    #the 6,5 hour returns. 
    dfwtw65  = pd.read_csv('dfwtw65.csv') 
    timeindicator2 = 13 #6,5 hours
    df17 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][0]) & (dfwtw['datetime'] <(dfwtw['datetime'][timeindicator2]))] #independent variable
    # df18 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator2]) & (dfwtw['datetime'] <(dfwtw['datetime'][13]))]
    df19 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator2]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-53]))].reset_index(drop=True) #dpeendend variables. #These should always have 13 because want the end of the first day.
    df20 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator2]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-40]))].reset_index(drop=True)
    df21 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator2]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-27]))].reset_index(drop=True)
    df22 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator2]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-14]))].reset_index(drop=True)
    df23 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator2]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-1]))].reset_index(drop=True)
    
    # print(dfwtw['datetime'][timeindicator2], dfwtw['datetime'].iloc[-53])
    # print(len(df17))
    # print(len(df19))
    # print(len(df20))
    # print(len(df21))
    # print(len(df22))
    # print(len(df23))
    # print(df17)
    # print(df19)
    # print(df20)
    # print(df21)
    # print(df22)
    # print(df23)
    
    
    sixfiveIVreturns = ((1 + df17['return']).cumprod() - 1)[timeindicator2-1]    
    # sixfiveDVreturn1 = ((1 + df18['return']).cumprod() - 1)[len(df18)-1] #cumprod until end of the first  day.
    sixfiveDVreturn2 = ((1 + df19['return']).cumprod() - 1)[len(df19)-1]# cumprod until the end of the second day enzovoorts. 
    sixfiveDVreturn3 = ((1 + df20['return']).cumprod() - 1)[len(df20)-1]
    sixfiveDVreturn4 = ((1 + df21['return']).cumprod() - 1)[len(df21)-1]
    sixfiveDVreturn5 = ((1 + df22['return']).cumprod() - 1)[len(df22)-1]
    sixfiveDVreturn6 = ((1 + df23['return']).cumprod() - 1)[len(df23)-1]

    adding3 = {'IV':sixfiveIVreturns, 'DV2':sixfiveDVreturn2,'DV3':sixfiveDVreturn3,'DV4':sixfiveDVreturn4,'DV5':sixfiveDVreturn5,'DV6': sixfiveDVreturn6, 'date': dateevent}
    dfwtw65 = dfwtw65.append(adding3, ignore_index=True) 
    dfwtw65.to_csv(Path(p, 'dfwtw65.csv'), index=False)

    #the 7 hour returns 
    dfwtw7  = pd.read_csv('dfwtw7.csv') 
    timeindicator3 = 14 #7 hours
    df24 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][0]) & (dfwtw['datetime'] <(dfwtw['datetime'][timeindicator3]))] #independent variable
    # df18 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator3]) & (dfwtw['datetime'] <(dfwtw['datetime'][13]))]
    df25 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator3]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-53]))].reset_index(drop=True)#dpeendend variables. #These should always have 13 because want the end of the first day.
    df26 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator3]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-40]))].reset_index(drop=True)
    df27 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator3]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-27]))].reset_index(drop=True)
    df28 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator3]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-14]))].reset_index(drop=True)
    df29 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator3]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-1]))].reset_index(drop=True)
    sevenIVreturns = ((1 + df24['return']).cumprod() - 1)[timeindicator3-1]    
    # sevenDVreturn1 = ((1 + df18['return']).cumprod() - 1)[len(df18)-1] #cumprod until end of the first  day.
    sevenDVreturn2 = ((1 + df25['return']).cumprod() - 1)[len(df25)-1]# cumprod until the end of the second day enzovoorts. 
    sevenDVreturn3 = ((1 + df26['return']).cumprod() - 1)[len(df26)-1]
    sevenDVreturn4 = ((1 + df27['return']).cumprod() - 1)[len(df27)-1]
    sevenDVreturn5 = ((1 + df28['return']).cumprod() - 1)[len(df28)-1]
    sevenDVreturn6 = ((1 + df29['return']).cumprod() - 1)[len(df29)-1]

    adding4 = {'IV':sevenIVreturns, 'DV2':sevenDVreturn2,'DV3':sevenDVreturn3,'DV4':sevenDVreturn4,'DV5':sevenDVreturn5,'DV6': sevenDVreturn6, 'date': dateevent}
    dfwtw7 = dfwtw7.append(adding4, ignore_index=True) 
    dfwtw7.to_csv(Path(p, 'dfwtw7.csv'), index=False)

    #first half hour last half hour. 
    dfhhr  = pd.read_csv('dfhhr.csv') 
    firsthalfhour = 1
    timeindicator4 = 12 #6 hours
    df30 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][0]) & (dfwtw['datetime'] <(dfwtw['datetime'][firsthalfhour]))].reset_index(drop=True) #independent variable
    df31 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][timeindicator4]) & (dfwtw['datetime'] <(dfwtw['datetime'][13]))].reset_index(drop=True)
    # print('print(df31')
    # print(df31) 
    df32 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'].iloc[-53]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-53]))].reset_index(drop=True)#dpeendend variables. #These should always have 13 because want the end of the first day.
    # print('print(df32')
    # print(df32) 
    df33 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'].iloc[-40]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-40]))].reset_index(drop=True)#
    df34 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'].iloc[-27]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-27]))].reset_index(drop=True)#
    df35 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'].iloc[-14]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-14]))].reset_index(drop=True)#
    df36 = dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'].iloc[-1]) & (dfwtw['datetime'] <=(dfwtw['datetime'].iloc[-1]))].reset_index(drop=True)
    # print( dfwtw.loc[(dfwtw['datetime'] >= dfwtw['datetime'][12]+timedelta(days=5))])
    # print('print(df36')
    # print(df36) 
    hhrIVreturns = ((1 + df30['return']).cumprod() - 1)[firsthalfhour-1]   
    # print(((1 + df31['return']).cumprod() - 1).iloc[0])
    # print(((1 + df31['return']).cumprod() - 1))
    # print(((1 + df31['return']).cumprod() - 1).values)
    hhrDVreturn1 = ((1 + df31['return']).cumprod() - 1)[len(df31)-1] #cumprod until end of the first  day.
    hhrDVreturn2 = ((1 + df32['return']).cumprod() - 1)[len(df32)-1]# cumprod until the end of the second day enzovoorts. 
    hhrDVreturn3 = ((1 + df33['return']).cumprod() - 1)[len(df33)-1]
    hhrDVreturn4 = ((1 + df34['return']).cumprod() - 1)[len(df34)-1]
    hhrDVreturn5 = ((1 + df35['return']).cumprod() - 1)[len(df35)-1]
    hhrDVreturn6 = ((1 + df36['return']).cumprod() - 1)[len(df36)-1]

    adding5 = {'IV':hhrIVreturns, 'DV': hhrDVreturn1,'DV2':hhrDVreturn2,'DV3':hhrDVreturn3,'DV4':hhrDVreturn4,'DV5':hhrDVreturn5,'DV6': hhrDVreturn6, 'date': dateevent}
    dfhhr = dfhhr.append(adding5, ignore_index=True) 
    dfhhr.to_csv(Path(p, 'dfhhr.csv'), index=False)

    #IV is first half hour returns 
    #Dv is last half hour returns, which are the same as dv2 first eod returns. 



    #always long benchmark whole day returns, although this cannot be used for the benchmark, this needs to be collected. 
    # df2.to_csv(Path(p, stockname + '_fiveminutedata.csv'))

In [183]:
fiveminohlcmaker('RUBI', 'wyqqpznupxkbdqr8', '2021-03-30 09:30:00')

0 bb works
58 bb works
0 rsi works
60 rsi works
14 ATR works


C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [178]:
last_row_remove('dfmr', 1)
last_row_remove('dfwtw6', 1)
last_row_remove('dfwtw65', 1)
last_row_remove('dfwtw7', 1)
last_row_remove('dfhhr', 1)

In [185]:
test = pd.read_csv('dfmr.csv')
test.tail()

,Unnamed: 0,IV,DV,DV2,DV3,DV4,DV5,DV6,date
235,NaN,-0.213029,-0.018159,0.028647,0.006544,0.055300,0.004161,-0.058247,2021-03-16 09:30:00
236,NaN,1.352294,0.418097,0.487239,0.380655,0.028861,0.107644,-0.023011,2021-03-23 09:30:00
237,NaN,-0.735740,-0.165798,-0.102179,-0.051095,-0.095172,-0.148071,-0.113660,2021-03-23 09:30:00
238,NaN,0.575532,0.077650,-0.174882,-0.250506,-0.299122,-0.282917,-0.311276,2021-03-30 09:30:00
239,NaN,-0.305791,0.211597,0.141404,0.243133,0.296033,0.275687,0.273652,2021-03-30 09:30:00


In [164]:
test = pd.read_csv('dfwtw6.csv')
test.tail()

,Unnamed: 0,IV,DV,DV2,DV3,DV4,DV5,DV6,date
223,NaN,1.122172,0.411514,-0.063966,-0.181237,-0.193987,-0.189765,-0.223284,2021-02-08 09:30:00
224,NaN,1.626667,-0.069374,-0.231760,-0.047360,-0.084602,0.064298,-0.065144,2021-02-17 09:30:00
225,NaN,6.621914,-0.152865,-0.466698,-0.441182,-0.505163,-0.554023,-0.620571,2021-02-16 09:30:00
226,NaN,0.923417,0.009488,0.096140,0.000823,-0.206447,-0.254798,-0.237468,2021-02-22 09:30:00
227,NaN,-0.424315,-0.034966,-0.071959,-0.054267,-0.051050,-0.060701,-0.070351,2021-02-22 09:30:00


In [350]:
test = pd.read_csv('dfwtw65.csv')
test.tail()

,Unnamed: 0,IV,DV2,DV3,DV4,DV5,DV6,date
33,NaN,1.788908,0.247126,0.155172,0.114943,-0.022989,0.014368,2017-04-11 09:30:00
34,NaN,1.285264,-0.185345,-0.327586,-0.378448,-0.380302,-0.361767,2017-04-07 09:30:00
35,NaN,-0.248976,0.063555,0.044218,-0.029448,0.005543,0.014752,2017-04-7 09:30:00
36,NaN,-0.406780,0.057143,-0.114286,-0.085714,-0.014286,0.028571,2017-04-13 09:30:00
37,NaN,2.006369,-0.269068,0.074153,-0.093220,-0.154661,-0.167373,2017-04-17 09:30:00


In [351]:
test = pd.read_csv('dfwtw7.csv')
test.tail()

,Unnamed: 0,IV,DV2,DV3,DV4,DV5,DV6,date
33,NaN,1.933162,1.857923e-01,0.098361,0.060109,-0.071038,-0.035519,2017-04-11 09:30:00
34,NaN,0.822301,2.162162e-02,-0.156757,-0.220541,-0.222865,-0.199622,2017-04-07 09:30:00
35,NaN,-0.233748,4.241877e-02,0.023466,-0.048736,-0.014440,-0.005415,2017-04-7 09:30:00
36,NaN,-0.372881,-3.330669e-16,-0.162162,-0.135135,-0.067568,-0.027027,2017-04-13 09:30:00
37,NaN,1.484076,-1.153846e-01,0.300000,0.097436,0.023077,0.007692,2017-04-17 09:30:00


In [75]:
test = pd.read_csv('dfhhr.csv')
test.tail()

,Unnamed: 0,IV,DV,DV2,DV3,DV4,DV5,DV6,date
82,NaN,1.227828,-0.003322,0.002045,0.000654,0.010849,0.006139,-0.003661,2017-09-05 09:30:00
83,NaN,-0.234104,-0.022857,-0.008347,0.003542,0.000370,0.000366,-0.001478,2017-09-05 09:30:00
84,NaN,1.041667,-0.029787,-0.024831,-0.019150,0.007246,0.000000,0.005102,2017-09-15 09:30:00
85,NaN,0.032727,-0.062500,-0.011236,-0.023256,0.000000,0.019048,-0.023810,2017-09-15 09:30:00
86,NaN,0.055971,0.009135,0.009132,-0.006120,0.042517,-0.011824,-0.037872,2017-09-15 09:30:00


In [108]:
import csv


def last_row_remove(df, numberremove):
    dffull = pd.read_csv(df+'.csv')
    dffull = dffull.iloc[:-numberremove]
    p = Path(r"C:\Users\david\Documents\universiteit\VU\Finance and Technology\Thesis")
    dffull.to_csv(Path(p, df+'.csv'), index=False)



In [550]:

df = pd.read_csv('CALA_fiveminutedata.csv', index_col=0) 
# df.index[0]+timedelta(days=1)
df
# df.reset_index(inplace=True)

# from scipy.stats import describe
# describe(df, axis=0)

# import pandas_profiling as pp
# eda = pp.ProfileReport(df)
# display(eda)

,datetime,SIZE,open,high,low,close,return,bollinger_down,sma,bollinger_up,rsi,atr1,bbI,rsiI,atrI,atrhigh,IndicatorMeanReversion
0,2021-01-04 09:30:00,1016941,2.500,2.7800,2.46,2.7000,-0.450102,4.263732,4.900288,5.536844,11.251902,0.202129,1,1,<ufunc 'absolute'>,0.202129,NaN
1,2021-01-04 09:35:00,387759,2.690,3.0200,2.69,3.0200,0.118519,4.033061,4.862508,5.691955,20.174164,0.224629,1,1,<ufunc 'absolute'>,0.224629,NaN
2,2021-01-04 09:40:00,951519,3.030,3.2000,2.85,2.9214,-0.032649,3.828421,4.822936,5.817451,19.556010,0.248200,1,1,<ufunc 'absolute'>,0.248200,NaN
3,2021-01-04 09:45:00,398384,2.935,2.9500,2.77,2.8250,-0.032998,3.637716,4.780858,5.924000,18.975942,0.259629,1,1,<ufunc 'absolute'>,0.259629,NaN
4,2021-01-04 09:50:00,300130,2.825,2.9400,2.77,2.9000,0.026549,3.480349,4.739858,5.999367,20.840013,0.271057,1,1,<ufunc 'absolute'>,0.271057,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,2021-01-11 15:35:00,4749,2.825,2.8300,2.82,2.8269,-0.001095,2.822804,2.885856,2.948908,49.106781,0.019714,1,1,1,0.304836,1.0
464,2021-01-11 15:40:00,49069,2.820,2.8500,2.82,2.8400,0.004634,2.821242,2.885356,2.949470,50.293198,0.020507,1,1,1,0.304836,1.0
465,2021-01-11 15:45:00,20498,2.840,2.8589,2.83,2.8499,0.003486,2.820408,2.885054,2.949700,51.170993,0.021857,1,1,1,0.304836,1.0
466,2021-01-11 15:50:00,29267,2.840,2.8500,2.83,2.8350,-0.005228,2.817896,2.884074,2.950252,49.819840,0.021143,1,1,1,0.304836,1.0


In [263]:
dateevent = '2021-01-04 09:30:00'
df = df.loc[df.index >= dateevent]
df = df.reset_index()

df['datetime'] = pd.to_datetime(df['datetime'], format= '%Y%m%d %H:%M:%S.%f')
df.dtypes



datetime          datetime64[ns]
SIZE                       int64
open                     float64
high                     float64
low                      float64
close                    float64
return                   float64
bollinger_down           float64
sma                      float64
bollinger_up             float64
rsi                      float64
atr1                     float64
dtype: object

In [280]:
# df.loc[(df['datetime'] > df['datetime'][0]) & (df['datetime'] <(df['datetime'][14]))]
df.loc[(df['datetime'] >= df['datetime'][14]) & (df['datetime'] <(df['datetime'][78]+timedelta(days=1)))]


,datetime,SIZE,open,high,low,close,return,bollinger_down,sma,bollinger_up,rsi,atr1,bbI,rsiI,atrI,atrhigh,IndicatorMeanReversion
14,2021-01-04 10:40:00,102256,2.8950,2.9500,2.8900,2.9500,0.017241,2.436251,4.334326,6.232401,25.576247,0.134121,1,1,1,0.304836,1.0
15,2021-01-04 10:45:00,141847,2.9483,2.9700,2.9400,2.9595,0.003220,2.365928,4.294434,6.222940,25.809584,0.112693,1,1,1,0.304836,1.0
16,2021-01-04 10:50:00,152755,2.9589,2.9800,2.9100,2.9100,-0.016726,2.295691,4.253534,6.211377,25.386401,0.092693,1,1,1,0.304836,1.0
17,2021-01-04 10:55:00,223812,2.9100,2.9199,2.8500,2.8700,-0.013746,2.226405,4.211734,6.197063,25.047755,0.084829,1,1,1,0.304836,1.0
18,2021-01-04 11:00:00,193837,2.8650,2.9039,2.8000,2.8385,-0.010976,2.158609,4.169504,6.180399,24.782107,0.080107,1,1,1,0.304836,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2021-01-05 15:35:00,29063,2.6700,2.6800,2.6607,2.6750,0.001873,2.581218,2.635262,2.689306,43.801616,0.018943,1,1,1,0.304836,1.0
152,2021-01-05 15:40:00,26594,2.6746,2.6800,2.6700,2.6700,-0.001869,2.583117,2.636762,2.690407,43.515834,0.018943,1,1,1,0.304836,1.0
153,2021-01-05 15:45:00,56412,2.6747,2.6800,2.6700,2.6800,0.003745,2.584421,2.638362,2.692303,44.258051,0.018586,1,1,1,0.304836,1.0
154,2021-01-05 15:50:00,101028,2.6750,2.7000,2.6700,2.7000,0.007463,2.587322,2.640862,2.694402,45.713837,0.018586,1,1,1,0.304836,1.0


In [268]:
#Make indicator colomns with for loops
df['bbI'] = np.abs
df['rsiI'] = np.abs
df['atrI'] = np.abs

#Cut the dataframe until where the data is the day of the event.
# df = df.loc[df.index > dateevent]

#reset index
# df = df.reset_index()

#input indicators in rows as 1. 
for i in range(len(df)):
    if df.low[i] <= df.bollinger_down[i]:
        df['bbI'].iloc[i:]=1
        break
for i in range(len(df)):
    if df.high[i] >= df.bollinger_up[i]:
        df['bbI'].iloc[i:]=1
        break        
for i in range(len(df)):
    if df.rsi[i] <= 15:
        df['rsiI'].iloc[i:]=1
        break        
for i in range(len(df)):
    if df.rsi[i] >= 85:
        df['rsiI'].iloc[i:]=1
        break


df['atrhigh'] = 0.0    
for i in range(len(df)):
    if df.atr1[i] > df.atrhigh.max():
        df['atrhigh'].iloc[i:]= df.atr1[i]
    elif df.atr1[i] <= 0.75*df.atrhigh.max():
        df['atrI'].iloc[i:] = 1
        break

df['IndicatorMeanReversion'] = np.nan
for i in range(len(df)):
    if df['bbI'][i] ==  df['rsiI'][i] == df['atrI'][i] == 1:
        df['IndicatorMeanReversion'].iloc[i:] = 1
        break

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [305]:

dfiv = df.loc[df['IndicatorMeanReversion'] != 1].copy()
dfiv['cumprod'] = np.nan
((1 + df['return'][:14]).cumprod() - 1)[13]


-0.4093686354378818

In [302]:
df.loc[df['IndicatorMeanReversion'] != 1]['return']

0    -0.450102
1     0.118519
2    -0.032649
3    -0.032998
4     0.026549
5     0.000000
6    -0.001724
7    -0.029326
8     0.008861
9     0.021164
10    0.024180
11    0.015177
12   -0.006645
13   -0.030100
Name: return, dtype: float64

In [303]:
df['return']

0     -0.450102
1      0.118519
2     -0.032649
3     -0.032998
4      0.026549
         ...   
463   -0.001095
464    0.004634
465    0.003486
466   -0.005228
467   -0.001764
Name: return, Length: 468, dtype: float64

In [135]:
dfmrtest= pd.DataFrame(columns=['IV', 'DV'])    
addingtest = {'IV':'1', 'DV': 2}
dfmrtest = dfmrtest.append(addingtest, ignore_index=True)

In [136]:
dfmrtest

,IV,DV
0,1,2


In [325]:
df = pd.read_csv('vlcydbsmt8dvfsgv.csv') 
df['DATETIME2']= np.nan
df['datetime']= np.nan
df['DATETIME2']= df["DATE"].astype(str) +' '+df["TIME_M"].astype(str)
df['datetime'] = pd.to_datetime(df['DATETIME2'], format= '%Y%m%d %H:%M:%S.%f')
df = df[df['SYM_SUFFIX'] !='Z']
print(df)
df2 = df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).sum()
df2['open']=np.nan
df2['high']=np.nan
df2['low']=np.nan
df2['close']=np.nan
print(df2)
df2['open']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).PRICE.first()
df2['high']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).PRICE.max()
df2['low']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).PRICE.min()
df2['close']=df.groupby(pd.Grouper(key='datetime', axis=0, freq='5min')).PRICE.last()
df2.drop(columns=['DATE','SYM_SUFFIX', 'PRICE'], axis=1, inplace=True)
df2.dropna(axis=0, inplace=True)
df2['return']=np.nan
# df2['return'] = np.log(df2['close'].shift(0))-np.log(df2['close'].shift(1))
# df2['return'][0] = np.log(df2['close'][0])-np.log(df2['open'][0])
df2['return'] = (df2['close'].shift(0)-df2['close'].shift(1))/df2['close'].shift(1)
df2['return'][0] = (df2['close'][0]-df2['open'][0])/df2['open'][0]
# df2.to_csv(r'C:\Users\david\Documents\universiteit\VU\Finance and

            DATE              TIME_M SYM_ROOT  SYM_SUFFIX  SIZE  PRICE  \
0       20201207   9:30:01.330681531     CALA         NaN  2295  4.730   
1       20201207   9:30:01.330692947     CALA         NaN  2295  4.730   
2       20201207   9:30:01.335229450     CALA         NaN     1  4.700   
3       20201207   9:30:02.701836846     CALA         NaN   100  4.710   
4       20201207   9:30:03.049993547     CALA         NaN   177  4.730   
...          ...                 ...      ...         ...   ...    ...   
189174  20210111  15:59:58.469052293     CALA         NaN   251  2.830   
189175  20210111  15:59:58.708804466     CALA         NaN   100  2.835   
189176  20210111  15:59:58.709633284     CALA         NaN   200  2.835   
189177  20210111  15:59:59.902283318     CALA         NaN   112  2.830   
189178  20210111  15:59:59.902285005     CALA         NaN   188  2.830   

                          DATETIME2                      datetime  
0        20201207 9:30:01.330681531 2020-12

C:\Users\david\AppData\Local\Temp/ipykernel_2892/1523581832.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [326]:
df2

,SIZE,open,high,low,close,return
datetime,,,,,,
2020-12-07 09:30:00,7770,4.7300,4.7900,4.70,4.7900,0.012685
2020-12-07 09:35:00,14525,4.7900,4.8400,4.79,4.8400,0.010438
2020-12-07 09:40:00,9196,4.8008,4.8100,4.77,4.7700,-0.014463
2020-12-07 09:45:00,3939,4.7750,4.8100,4.74,4.8100,0.008386
2020-12-07 09:50:00,733,4.7900,4.8200,4.78,4.8000,-0.002079
...,...,...,...,...,...,...
2021-01-11 15:35:00,4749,2.8250,2.8300,2.82,2.8269,-0.001095
2021-01-11 15:40:00,49069,2.8200,2.8500,2.82,2.8400,0.004634
2021-01-11 15:45:00,20498,2.8400,2.8589,2.83,2.8499,0.003486
